# Notebook to fit persistence in the darks


In [ ]:
from astropy.io import fits
import glob, os, shutil, pickle, bz2, gc
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.stats import sigmaclip
from scipy.optimize import curve_fit,minimize
from scipy.special import gammaincc, gamma
from scipy.interpolate import interp1d
from astropy.wcs import WCS
from astropy.stats import histogram
from itertools import product
from multiprocessing import Pool

%matplotlib notebook

In [ ]:
pwd

In [ ]:
# The project dir 
pdir = '/user/gennaro/Functional_work/WFC3_persistence/py_progs/short_term_persistence/'

#The mosaic dir
mdir = pdir+'/Mosaic_hi_res_folder/'

#The dir to save/load the Persistence curves dataframes
sdir = pdir+'/PD_dataframes_dir/'


In [ ]:
# Read the file that contains the drak expsoure persistence measurements

with open(sdir+'/Mean_res_dark_only.pickle', 'rb') as f:
    mean_res_drk = pickle.load(f)


In [ ]:
# The integrated A-gamma-shift model.
# A is the prediction at 1000 seconds


def intpwl(t,pars):
    
    A = pars[0] 
    index = pars[1]
    
    tu = t[1:]
    td = t[:-1]
    if (index == -1.):
        return A*np.log( tu/td )
    else:
        return A/(1+index) * ( (tu/1000)**(1+index) - (td/1000)**(1+index) )
    
def intshpwl(t,pars):
    
    A = pars[0] 
    index = pars[1]
    t0 = pars[2]
    
    tu = t[1:]
    td = t[:-1]
    if (index == -1.):
        return A*np.log( (tu+t0)/(td+t0) )
    else:
        return A/(1+index) * ( ((tu+t0)/1000)**(1+index) - ((td+t0)/1000)**(1+index) )

def intdexp1(t,pars):
    
    a1 = pars[0]
    t1 = pars[1]
    
    tu = t[1:]
    td = t[:-1]
    k  = -a1*t1
    return k*(np.exp(-tu/t1)-np.exp(-td/t1))
    
def intexp2(t,pars):

    a1 = pars[0]
    t1 = pars[1]
    a2 = pars[2]
    t2 = pars[3]

    tu = t[1:]
    td = t[:-1]
    k1,k2  = -a1*t1, - a2*t2
    
    if (t1< 0) | (t2 < 0) | (a1< 0) | (a2 < 0)| (t1> 500) | (t2 < 500):
        return -np.inf
    else:
        return k1*(np.exp(-tu/t1)-np.exp(-td/t1)) + k2*(np.exp(-tu/t2)-np.exp(-td/t2))

def intexpN(t,pars):

    ncomp = len(pars)//2
    tu = t[1:]
    td = t[:-1]
    
    res = np.zeros(len(tu))
    
    for i in range(ncomp):
        ai = pars[2*i]
        ti = pars[2*i+1]
        ki  = -ai*ti
    
        if (ti< 0) |  (ai < 0):
            return -np.inf
        else:
            res = res+ki*(np.exp(-tu/ti)-np.exp(-td/ti))
        
    return res

    
def Agamma_predictions(pars,funcname,ilev,lthr,hthr,i2fl):

    residuals = []
    for j,i2f in enumerate(i2fl):
        ct = mean_res_drk['med_times'][ilev][i2f]
        pers = mean_res_drk['med_pers'][ilev][i2f]
        
        semi_int = 0.5*intervals[j]
        for c,p in zip(ct,pers):
            if (c > lthr) & (c < hthr):
                t = np.array([c-semi_int,c+semi_int])
                mod_pred = funcname(t,pars)/(2*semi_int)
            
                #print(t,A,index,mod_pred)
                residuals.append(np.square(mod_pred-p))

    return np.sum(residuals)
    
#function to subtract a gamm=-1 trend for the purpose of plotting
def Agamma_ref(t,A=300,gamma=-1):
    return A* np.power(t,gamma)


In [ ]:
v_name_to_fit = []

for vv in ['VIS1','VIS2','VIS3','ALL']:
    name_to_fit = [] 
    for mrd in mean_res_drk['Readme']:
        t1 = vv in mrd
        t2 = '_PDRK_S1' in mrd
        if t1&t2:
            name_to_fit.append(mrd)
    v_name_to_fit.append(name_to_fit)
            
v_ind_to_fit = [[mean_res_drk['Readme'].index(name) for name in name_to_fit] for name_to_fit in v_name_to_fit]

intervals = [25.,100.,2.9,12.5,50]

for name_to_fit,ind_to_fit in zip(v_name_to_fit,v_ind_to_fit):
    for n,i,it in zip(name_to_fit,intervals,ind_to_fit):
        print(n,i,it)

    
funcname = intpwl
pars0 = np.array([2000,-.6])  # good for intpwl
#pars0 = np.array([5,150,1,1500])  # good for intexp2
#pars0 = np.array([10,100,10,500,10,1000])  # good for intexpN with N=3



In [ ]:
plt.style.use('bmh')
plt.rcParams['font.family'] = 'Times New Roman'
plt.rcParams['font.size'] = 15
plt.rcParams['axes.labelsize'] = 15
plt.rcParams['axes.labelweight'] = 'normal'
plt.rcParams['xtick.labelsize'] = 14
plt.rcParams['ytick.labelsize'] = 14
plt.rcParams['legend.fontsize'] = 12
plt.rcParams['figure.titlesize'] = 16

In [ ]:

#lthrs = [0 for i in range(12)]
#hthrs = [200*(1.25**i) for i in range(len(lthrs))]

#lthrs = [0.,0.,0.,0.,0.]
#hthrs = [250.,500.,550.,750.,2000.]

#lthrs = [0.,0.,0.]
#hthrs = [2000.,4000.,6000.]

lthrs = [0.,0.,0.]
hthrs = [250,1000.,4000]

print(lthrs)
print(hthrs)

loclist = [np.power(0.7,i) for i in range(-9,9,1)]
formlist = ['{:3.2f}'.format(ll) for ll in loclist]


BF_res = np.empty([len(pars0),len(mean_res_drk['levels'])-1,len(v_ind_to_fit),len(lthrs)]) 

import matplotlib

clist = ['Reds','Greens','Blues','Greys']
llist = ['VIS1','VIS2','VIS3','ALL']

for ilev in range(len(mean_res_drk['levels'])-1):

    fig, ax = plt.subplots(len(v_ind_to_fit),len(v_ind_to_fit[0]), figsize=(2.75*len(v_ind_to_fit[0]),1.875*len(v_ind_to_fit)))
    meanlev = 0.5*(mean_res_drk['levels'][ilev]+mean_res_drk['levels'][ilev+1])
    fig.suptitle('Stimulus = {:6.2f}'.format(10**meanlev/80000.) +' x Saturation',fontsize=18)
    
    for zz,ind_to_fit in enumerate(v_ind_to_fit):

        cmap = matplotlib.cm.get_cmap(clist[zz])
        colors = [cmap(0.1+0.7*(i+1)/(1*len(lthrs))) for i in range(len(lthrs))]

        for w,(lthr,hthr,color) in enumerate(zip(lthrs,hthrs,colors)):
    
            bfres = minimize(Agamma_predictions, pars0, args=(funcname,ilev,lthr,hthr,ind_to_fit), method='Powell')
            if(bfres['status'] == 0):
                print('Best fit results for level {:6.2f} and fit time from {:7.1f} to {:7.1f} seconds and visit {:d}'.format(10**meanlev/80000.,lthr,hthr,zz+1))
                print(bfres['x'])
            else:
                print('Fit failed for level {:6.2f} and fit time from {:7.1f} to {:7.1f} seconds and visit {:d}'.format(10**meanlev/80000.,lthr,hthr,zz+1))
            
            BF_res[:,ilev,zz,w] = bfres['x']
            
            for j,i2f in enumerate(ind_to_fit):
                ct = mean_res_drk['med_times'][ilev][i2f]
                pers = mean_res_drk['med_pers'][ilev][i2f]

                semi_int = 0.5*intervals[j]
                mod_pred = np.empty_like(pers)

                for k,(c,p) in enumerate(zip(ct,pers)):
                    t = np.array([c-semi_int,c+semi_int])
                    mod_pred[k] = funcname(t,bfres['x'])/(2*semi_int)  

                ax[zz,j].plot(ct,mod_pred/Agamma_ref(ct),c=color)
                ax[zz,j].set_axis_bgcolor('#FFFFFF')
                ax[zz,j].axvline(hthr*(1.04**zz),linestyle='--',c=color)

                if w == (len(lthrs)-1):
                    #ttt = np.array([1,10000])
                    ax[zz,j].scatter(ct,pers/Agamma_ref(ct),alpha=0.6,s=20,label=llist[zz],edgecolors=cmap(0.9),facecolors=cmap(0.8),linewidth=1)
                    ax[zz,j].set_xscale('log')
                    ax[zz,j].set_yscale('log')
                    #ax[zz,j].plot(ttt,np.array([0,0]),':',c='black')
                    #ax[ilev,j].plot(ct,Agamma_ref(ct,gamma=-0.9)/Agamma_ref(ct),'--',c='black')
                    #ax[ilev,j].plot(ct,Agamma_ref(ct,gamma=-1.1)/Agamma_ref(ct),c='black')                    
                    if ilev in [0,1]:
                        ax[zz,j].set_ylim(0.09*(ilev+1),0.4*(ilev+1))
                    elif ilev in [2]:
                        ax[zz,j].set_ylim(0.7,1.9)
                    elif ilev in [3,4]:
                        ax[zz,j].set_ylim(0.9,2.)
                    elif ilev in [5,6]:
                        ax[zz,j].set_ylim(0.99,2.9)
                    elif ilev in [7,8]:
                        ax[zz,j].set_ylim(1.15,2.95)                        
                    else:
                        ax[zz,j].set_ylim(1.35,3.)
                    
                    ax[zz,j].set_xlim(85,5000)
                    if zz ==0:
                        ax[zz,j].set_title('$\delta t$ = {:5.1f} [s]'.format(intervals[j]),fontsize=16)
                    
                    if zz != (len(v_ind_to_fit)-1):
                        ax[zz,j].axes.xaxis.set_ticklabels([])

                    if j!=0:
                        ax[zz,j].yaxis.set_major_locator(plt.FixedLocator(loclist))
                        ax[zz,j].yaxis.set_major_formatter(plt.NullFormatter())    
                        ax[zz,j].yaxis.set_minor_formatter(plt.NullFormatter())   
                    else:
                        ax[zz,j].legend(loc=2)
                        ax[zz,j].yaxis.set_major_locator(plt.FixedLocator(loclist))
                        ax[zz,j].yaxis.set_major_formatter(plt.FixedFormatter(formlist)) 
                        ax[zz,j].yaxis.set_minor_formatter(plt.NullFormatter()) 

    
    fig.text(0.5, 0.02, r'Time since end of stimulus [s]', ha='center', fontsize=17)
    fig.text(0.02, 0.5, r'Persistence/P$_{\mathrm{ref}}$', va='center', rotation='vertical',fontsize=17)
    fig.tight_layout(rect=[0.04, 0.04, 1, 0.95],w_pad=0.1,h_pad=0.3)
    fig.savefig('/user/gennaro/Functional_work/ISRs/Short_term_persistence_I/Figures/PersDecay_{:06.2f}'.format(10**meanlev/80000.) +'.pdf')
    

In [ ]:
fig,ax = plt.subplots(2,6,figsize=(15,5))

#BF_res = np.empty([len(pars0),len(mean_res_drk['levels'])-1,len(v_ind_to_fit)]) 
#hthrs = [250,1000.,4000]

loclist0 = [0.125*np.power(4,i) for i in range (15)]
formlist0 = ['{:3.2f}'.format(ll) for ll in loclist0]

loclist1 = [0.125*np.power(2,i) for i in range (15)]
formlist1 = ['{:3.2f}'.format(ll) for ll in loclist1]

loclistA = [0.512*np.power(1/0.8,i) for i in range (20)]
formlistA = ['{:3.2f}'.format(ll) for ll in loclistA]

for ilev in range(len(mean_res_drk['levels'])-1):

    ix = ilev // 6
    iy = ilev % 6
    
    meanlev = 0.5*(mean_res_drk['levels'][ilev]+mean_res_drk['levels'][ilev+1])
    ax[ix,iy].set_title('Stim. = {:6.2f}'.format(10**meanlev/80000.) +' x sat.',fontsize=15)
    ax[ix,iy].set_xscale('log')
    ax[ix,iy].set_yscale('log')
#    ax[ix,iy].set_ylim(0.01,5)
    ax[ix,iy].set_xlim(85,5000)

    for zz,ind_to_fit in enumerate(v_ind_to_fit):
        cmap = matplotlib.cm.get_cmap(clist[zz])
        pers = np.empty(len(hthrs))
        for w,hthr in enumerate(hthrs):
            pers[w] = BF_res[0,ilev,zz,w]*np.power(hthrs[w],BF_res[1,ilev,zz,w])
        ax[ix,iy].plot(hthrs,pers/Agamma_ref(hthrs,A=BF_res[0,ilev,zz,0],gamma=BF_res[1,ilev,zz,0]),'o--',color=cmap(0.9),label=llist[zz])
    
#    if iy!=0:
#        print('ciao')
#        ax[ix,iy].yaxis.set_major_formatter(plt.NullFormatter())    
#        ax[ix,iy].yaxis.set_minor_formatter(plt.NullFormatter())   
#    else:
#        ax[ix,iy].legend(loc=2)
    if ilev in [0]:
        ax[ix,iy].yaxis.set_major_locator(plt.FixedLocator(loclist0))
        ax[ix,iy].yaxis.set_major_formatter(plt.FixedFormatter(formlist0)) 
    elif ilev in [1]:
        ax[ix,iy].yaxis.set_major_locator(plt.FixedLocator(loclist1))
        ax[ix,iy].yaxis.set_major_formatter(plt.FixedFormatter(formlist1))     
    else:
        ax[ix,iy].yaxis.set_major_locator(plt.FixedLocator(loclistA))
        ax[ix,iy].yaxis.set_major_formatter(plt.FixedFormatter(formlistA)) 
    
    if ilev == 5:
         ax[ix,iy].legend(fontsize=10)
            
    ax[ix,iy].yaxis.set_minor_formatter(plt.NullFormatter()) 

        #ax[zz,j].yaxis.set_major_locator(plt.FixedLocator([0.0625,0.125,0.25,0.5,1,2,4,8]))
        #ax[zz,j].yaxis.set_major_formatter(plt.FixedFormatter(['0.0625','0.125','0.25','0.5','1','2','4','8'])) 
    
#    ax[ix,iy].yaxis.set_major_locator(plt.FixedLocator([0.1,1,10]))
    ax[ix,iy].set_axis_bgcolor('#FFFFFF')

fig.text(0.5, 0.02, r'Time since end of stimulus [s]', ha='center', fontsize=17)
fig.text(0.02, 0.5, r'Persistence/P$_{250}$', va='center', rotation='vertical',fontsize=17)
fig.tight_layout(rect=[0.04, 0.04, 1, 0.95],w_pad=0.5,h_pad=0.4)
fig.savefig('/user/gennaro/Functional_work/ISRs/Short_term_persistence_I/Figures/P_over_P250.pdf')


In [ ]:
fig,ax = plt.subplots(2,6,figsize=(15,5))

#BF_res = np.empty([len(pars0),len(mean_res_drk['levels'])-1,len(v_ind_to_fit)]) 
#hthrs = [250,1000.,4000]


for ilev in range(len(mean_res_drk['levels'])-1):

    ix = ilev // 6
    iy = ilev % 6
    
    meanlev = 0.5*(mean_res_drk['levels'][ilev]+mean_res_drk['levels'][ilev+1])
    ax[ix,iy].set_title('Stim. = {:6.2f}'.format(10**meanlev/80000.) +' x sat.',fontsize=15)
    ax[ix,iy].set_xscale('log')
#    ax[ix,iy].set_ylim(0.01,5)
    ax[ix,iy].set_xlim(85,5000)

    for zz,ind_to_fit in enumerate(v_ind_to_fit):
        cmap = matplotlib.cm.get_cmap(clist[zz])
        pers = np.empty(len(hthrs))
        ax[ix,iy].plot(hthrs,BF_res[1,ilev,zz,:],'o--',color=cmap(0.9),label=llist[zz])
    
    
    if ilev == 5:
         ax[ix,iy].legend(fontsize=10)
            
#    ax[ix,iy].yaxis.set_minor_formatter(plt.NullFormatter()) 

        #ax[zz,j].yaxis.set_major_locator(plt.FixedLocator([0.0625,0.125,0.25,0.5,1,2,4,8]))
        #ax[zz,j].yaxis.set_major_formatter(plt.FixedFormatter(['0.0625','0.125','0.25','0.5','1','2','4','8'])) 
    
#    ax[ix,iy].yaxis.set_major_locator(plt.FixedLocator([0.1,1,10]))
    ax[ix,iy].set_axis_bgcolor('#FFFFFF')

fig.text(0.5, 0.02, r'Time since end of stimulus [s]', ha='center', fontsize=17)
fig.text(0.02, 0.5, r'$\gamma$', va='center', rotation='vertical',fontsize=17)
fig.tight_layout(rect=[0.04, 0.04, 1, 0.95],w_pad=0.5,h_pad=0.4)

fig.savefig('/user/gennaro/Functional_work/ISRs/Short_term_persistence_I/Figures/gammas.pdf')


In [ ]:
fig,ax = plt.subplots(2,6,figsize=(15,5))

#BF_res = np.empty([len(pars0),len(mean_res_drk['levels'])-1,len(v_ind_to_fit)]) 
#hthrs = [250,1000.,4000]


for ilev in range(len(mean_res_drk['levels'])-1):

    ix = ilev // 6
    iy = ilev % 6
    
    meanlev = 0.5*(mean_res_drk['levels'][ilev]+mean_res_drk['levels'][ilev+1])
    ax[ix,iy].set_title('Stim. = {:6.2f}'.format(10**meanlev/80000.) +' x sat.',fontsize=15)
    ax[ix,iy].set_xscale('log')
#    ax[ix,iy].set_ylim(0.01,5)
    ax[ix,iy].set_xlim(85,5000)

    for zz,ind_to_fit in enumerate(v_ind_to_fit):
        cmap = matplotlib.cm.get_cmap(clist[zz])
        pers = np.empty(len(hthrs))
        ax[ix,iy].plot(hthrs,BF_res[0,ilev,zz,:],'o--',color=cmap(0.9),label=llist[zz])
    
    
    if ilev == 5:
         ax[ix,iy].legend(fontsize=10)
            
#    ax[ix,iy].yaxis.set_minor_formatter(plt.NullFormatter()) 

        #ax[zz,j].yaxis.set_major_locator(plt.FixedLocator([0.0625,0.125,0.25,0.5,1,2,4,8]))
        #ax[zz,j].yaxis.set_major_formatter(plt.FixedFormatter(['0.0625','0.125','0.25','0.5','1','2','4','8'])) 
    
#    ax[ix,iy].yaxis.set_major_locator(plt.FixedLocator([0.1,1,10]))
    ax[ix,iy].set_axis_bgcolor('#FFFFFF')

fig.text(0.5, 0.02, r'Time since end of stimulus [s]', ha='center', fontsize=17)
fig.text(0.02, 0.5, r'A', va='center', rotation='vertical',fontsize=17)
fig.tight_layout(rect=[0.04, 0.04, 1, 0.95],w_pad=0.5,h_pad=0.4)

fig.savefig('/user/gennaro/Functional_work/ISRs/Short_term_persistence_I/Figures/As.pdf')
